In this series of notebooks, I will attempt to do some introductory exploration of various roller derby statistics. We will use the publicly available stats on the FlatTrackStats website. First, I will build a table scraper tool using the BeautifulSoup4 package to parse the stats tables on the website. If not already installed, you will need pandas and BeautifulSoup4 in order to run this notebook. 

In [23]:
    import requests
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    from itertools import product
    from urllib.request import urlopen
    import re
    
    #First, define a class to parse HTML tables for bouts and players
    
    class HTMLTableParser:
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(self.read_table(table)) for table in soup.find_all('table')]      
        
        def read_table(self, table_tag):
            rowspans = []  # track pending rowspans
            rows = table_tag.find_all('tr')

            # first scan, see how many columns we need
            colcount = 0
            column_names=[]
            for r, row in enumerate(rows):
                cells = row.find_all(['td', 'th'], recursive=False)
                colcount = max(
                    colcount,
                    sum(int(c.get('colspan', 1)) or 1 for c in cells[:-1]) + len(cells[-1:]) + len(rowspans))
                # update rowspan bookkeeping; 0 is a span to the bottom. 
                rowspans += [int(c.get('rowspan', 1)) or len(rows) - r for c in cells]
                rowspans = [s - 1 for s in rowspans if s > 1]

            # it doesn't matter if there are still rowspan numbers 'active'; no extra
            # rows to show in the table means the larger than 1 rowspan numbers in the
            # last table row are ignored.    
            # build an empty matrix for all possible cells
            table = [[None] * colcount for row in rows]


            # fill matrix from row data
            rowspans = {}  # track pending rowspans, column number mapping to count
            for row, row_elem in enumerate(rows):
                span_offset = 0  # how many columns are skipped due to row and colspans 
                for col, cell in enumerate(row_elem.find_all(['td', 'th'], recursive=False)):
                    # adjust for preceding row and colspans
                    col += span_offset
                    while rowspans.get(col, 0):
                        span_offset += 1
                        col += 1

                    # fill table data
                    rowspan = rowspans[col] = int(cell.get('rowspan', 1)) or len(rows) - row
                    colspan = int(cell.get('colspan', 1)) or colcount - col
                    # next column is offset by the colspan
                    span_offset += colspan - 1
                    value = cell.get_text()
                    points = len(cell.find_all("div", {"class": " point"}))
                    pens = cell.find_all("div", {"class": "penalty major"})  
                    if (value == '' or value == '&nbsp') and points != '':
                        value = str(points)
                    if len(pens) > 0:
                        value = ''
                        for pen in pens:
                            value += pen.find_all(text=True, recursive=False)[0] + " "
                    if len(cell.find_all("div", {"class": "lead"}))== 1:
                        if value == "0": value = ""
                        value += "Lead "
                    if len(cell.find_all("div", {"class": "leadloss"}))== 1:
                        if value == "0": value = ""
                        value += "LeadLoss "
                    if len(cell.find_all("div", {"class": "lost"}))== 1:
                        if value == "0": value = ""
                        value += "LeadLoss "                   
                    if len(cell.find_all("div", {"class": "call"}))== 1:
                        if value == "0": value = ""
                        value += "call "
                    if len(cell.find_all("div", {"class": "nopass"}))== 1:
                        if value == "0": value = ""
                        value += "np "    
                    if len(cell.find_all("div", {"class": "starpass"}))== 1:
                        if value == "0": value = ""
                        value += "sp "    
                    for drow, dcol in product(range(rowspan), range(colspan)):
                        try:
                            table[row + drow][col + dcol] = value
                            rowspans[col + dcol] = rowspan
                        except IndexError:
                            # rowspan or colspan outside the confines of the table
                            pass

                # update rowspan bookkeeping
                rowspans = {c: s - 1 for c, s in rowspans.items() if s > 1}
            npt = np.array(table)
            #df = pd.DataFrame(np.array(table), column_names) 
            return table

        def parse_url_todf(self, url):
            tables = self.parse_url(url)
            dfs = []
            if "combos" in url:
                for table in tables:
                    headers = table.pop(0)
                    df = pd.DataFrame(np.array(table), columns=np.array(headers))    
                    dfs.append(df)
            if "teams" in url:
                for table in tables:
                    headers = table.pop(0)
                    df = pd.DataFrame(np.array(table), columns=np.array(headers))    
                    dfs.append(df)
            if "jams" in url:
                for table in tables:
                    headers = table.pop(0)
                    df = pd.DataFrame(np.array(table), columns=np.array(headers))    
                    dfs.append(df)        
            else:
                headers = tables[0]
                headersnew = []
                for header in headers:
                    headernew = [i for i in header if i] 
                    headersnew.append(headernew)
                for i in range(len(tables)-1):
                    df = pd.DataFrame(np.array(tables[i+1]), columns=headersnew[i])    
                    dfs.append(df)
                
            return dfs    

In [24]:
def GetAllBouts(teamID):
    npages = 0
    links=[]
    
    base_url = "http://flattrackstats.com/teams/"+teamID+"/bouts"
    text = urlopen(base_url).read()
    base_soup = BeautifulSoup(text)
    
    for listitem in base_soup.findAll('li', class_="pager-last last"):
        npages = int(listitem.findAll('a')[0]['href'][-1])
    
    
    for page in range(0,npages):
        url = "http://flattrackstats.com/teams/3393/bouts?page="+str(page)
        text = urlopen(url).read()
        soup = BeautifulSoup(text)
        for link in soup.findAll('a', class_="boutlink has-stats"):
            if "node" not in link['href']:
                linkname = link['href']
                linksize = len(linkname)
                linkstub = linkname[:linksize-8]
                links.append(linkstub)
         
    return links

In [25]:
GetAllBouts(str(3393))

['/bouts/89967/',
 '/bouts/89829/',
 '/bouts/89827/',
 '/bouts/89825/',
 '/bouts/88760/',
 '/bouts/89826/',
 '/bouts/89082/',
 '/bouts/89828/',
 '/bouts/89490/',
 '/bouts/89485/',
 '/bouts/89481/',
 '/bouts/89822/',
 '/bouts/76966/',
 '/bouts/74386/',
 '/bouts/76965/',
 '/bouts/76964/',
 '/bouts/74481/',
 '/bouts/76963/',
 '/bouts/76962/',
 '/bouts/76742/',
 '/bouts/75341/',
 '/bouts/68648/',
 '/bouts/67875/',
 '/bouts/60955/',
 '/bouts/60970/',
 '/bouts/58695/',
 '/bouts/60968/',
 '/bouts/60388/',
 '/bouts/60483/',
 '/bouts/60953/',
 '/bouts/60964/',
 '/bouts/59216/',
 '/bouts/60951/',
 '/bouts/60962/',
 '/bouts/60949/',
 '/bouts/60957/',
 '/bouts/59173/',
 '/bouts/60638/',
 '/bouts/43604/',
 '/bouts/46000/',
 '/bouts/49725/',
 '/bouts/48932/',
 '/bouts/46004/',
 '/bouts/45257/',
 '/bouts/26993/',
 '/bouts/26751/',
 '/bouts/26996/',
 '/bouts/28453/',
 '/bouts/20826/',
 '/bouts/8857/',
 '/bouts/8219/',
 '/bouts/8500/',
 '/bouts/8250/',
 '/bouts/6924/',
 '/bouts/6164/',
 '/bouts/6158/',

In [26]:
def GetAllScores(teamID, teamName):
    scoreframes = []
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for bout in bouts:
        jampage = "http://flattrackstats.com"+bout+"jams"
        
        isAway = False 
        response = requests.get(jampage)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            substr = fulltitle[fulltitle.find(':')+1 : fulltitle.find('(')]
            if teamName in substr:
                isAway = True
        
        scores = hp.parse_url_todf(jampage)[0].iloc[:,1:-1]
        scores.loc[np.c_[scores.index[1::2].tolist(),scores.index[0:-1:2].tolist()].reshape(-1)]
        
        #del scores[0]
        #print(scores)
        scoreframes.append(scores)
    allScores = pd.concat(scoreframes)
    #print(allScores)
    return(allScores)

In [50]:
def GetAllLineups(teamID, teamName):
    lineupframes = []
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for bout in bouts:
        jampage = "http://flattrackstats.com"+bout+"jams"
        
        isAway = False 
        response = requests.get(jampage)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            substr = fulltitle[fulltitle.find(':')+1 : fulltitle.find('(')]
            if teamName in substr:
                isAway = True
                
        lineups = hp.parse_url_todf(jampage)[1].iloc[:,1:-1]
        
        #put team of interest first (data is naturally in format (home, away) )
        if isAway:
            lineups.loc[np.c_[lineups.index[1::2].tolist(),lineups.index[0:-1:2].tolist()].reshape(-1)]
        
        lineupframes.append(lineups) 
        
    #clean data    
    allLineups = pd.concat(lineupframes)
    allLineups.columns = ['Jammer','Jstats','B1','B2','B3','B4','NULL','jamscore','runscore']
    del allLineups['NULL']
    
    allLineups = allLineups[~(allLineups.jamscore == 'Period 2')]
    allLineups = allLineups[~(allLineups.jamscore.isnull())]
    
    #calc score differentials, for determining rankings
    allLineups['jamscore'] = pd.to_numeric(allLineups['jamscore'])
    allLineups['runscore'] = pd.to_numeric(allLineups['runscore'])
    
    allLineups['ScoreDiff'] = allLineups['jamscore'].diff(periods=-1)
    allLineups.loc[1::2,"ScoreDiff"] = allLineups['jamscore'].diff(periods=1)
    
    #drop opposing team
    allLineups = allLineups.iloc[::2]
    allLineups = allLineups.reset_index()
    del allLineups['index']
    
    return(allLineups)

In [51]:
def GetAllPenalties(teamID, teamName):
    penframes = []
    bouts = GetAllBouts(teamID)
    hp = HTMLTableParser()
    for bout in bouts:
        jampage = "http://flattrackstats.com"+bout+"jams"
        
        isAway = False 
        response = requests.get(jampage)
        soup = BeautifulSoup(response.text, 'lxml')
        for link in soup.findAll('title'):
            fulltitle = link.get_text()
            substr = fulltitle[fulltitle.find(':')+1 : fulltitle.find('(')]
            if teamName in substr:
                isAway = True
        
        pens = hp.parse_url_todf(jampage)[2].iloc[:,1:-1]
        
        if isAway:
            pens.loc[np.c_[pens.index[1::2].tolist(),pens.index[0:-1:2].tolist()].reshape(-1)]

        #del pens[0]
        penframes.append(scores)
    allPens = pd.concat(penframes)
    #print(allPens)
    return(allPens)

In [ ]:
GetAllLineups(str(3393), 'Ohio')